In [ ]:
import os
import random
import numpy as np
import imageio
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, Flatten, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
%matplotlib inline  

In [ ]:
## Load dataset

In [ ]:
def load_dataset(path):
    files = os.listdir(path)
    x = []
    for file in files: # number of files to go through
        im = imageio.imread(os.path.join(path, file))
        x.append(im)
    x = np.asarray(x)
    return x

In [ ]:
path_train_orig = './Food11_256/training_256'
path_val_orig = './Food11_256/validation_256'

x_train = load_dataset(path_train_orig)
x_val = load_dataset(path_val_orig)

print('original dataset:', np.shape(x_train), np.shape(x_val))

In [ ]:
path_train_date = './Food11_256/training_256_datestamp'
path_val_date = './Food11_256/validation_256_datestamp'

x_train_date = load_dataset(path_train_date)
x_val_date = load_dataset(path_val_date)

print('datestamped dataset:', np.shape(x_train_date), np.shape(x_val_date))

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_val  = x_val.astype('float32') / 255.0
x_train_date = x_train_date.astype('float32') / 255.0
x_val_date  = x_val_date.astype('float32') / 255.0

In [ ]:
## Convolutional AutoEncoder

In [ ]:
epochs = 100
batch_size = 16
in_shape = (x_train.shape[1:])

In [ ]:
input_img = Input(shape=(in_shape)) # 1st layer 
x = Conv2D(96, (3, 3), padding='same')(input_img)  #2nd layer 
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)  # 3rd layer 
x = Conv2D(48, (3, 3), padding='same')(x)   #4th layer
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(48, (3, 3), padding='same')(encoded) # 7th layer
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)      # 8th layer 
x = Conv2D(96, (3, 3), padding='same')(x)  # 9th layer 
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x) # 10th layer 
x = Conv2D(3, (3, 3), padding='same')(x)  # 13rd 
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [ ]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

In [ ]:
history = model.fit(x_train_date, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val_date, x_val),
                    #callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')],
                    #callbacks=[TensorBoard(log_dir='date_stamp', histogram_freq=0, write_grads=False)],
                    shuffle=True
                   )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
## Visualize original image and reconstructed image

In [ ]:
# definition to show original image and reconstructed image
def showOrigDec(orig, dec, num=9):
    n = num
    plt.figure(figsize=(23, 5))

    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[i].reshape(in_shape))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[i].reshape(in_shape))
        ax.get_xaxis().set_visible(False)  
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
def random_subset(iterator, K):
    result = []
    N = 0

    for item in iterator:
        N += 1
        if len(result) < K:
            result.append(item)
        else:
            s = int(random.random() * N)
            if s < K:
                result[ s ] = item

    return result

In [ ]:
import math

def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [ ]:
path_test_orig = './Food11_256/evaluation_256'
x_test = load_dataset(path_test_orig)
x_test  = x_test.astype('float32') / 255.0

path_test_date = './Food11_256/evaluation_256_datestamp'
x_test_date = load_dataset(path_test_date)
x_test_date  = x_test_date.astype('float32') / 255.0

In [ ]:
d_sum = 0
for i in range(len(x_test)):
    dateimg = np.asarray([x_test_date[i]])
    nodateimg = np.asarray([x_test[i]])
    predictions = model.predict(dateimg)
    d=psnr(predictions,nodateimg)
    d_sum+=d
#         showOrigDec(nodateimg,predictions,1)
#         print("psnr:",d)
print(d_sum/len(x_test))

In [ ]:
n = 9
samples = np.asarray(random_subset(x_test_date, n))
predictions = model.predict(samples)
showOrigDec(samples,predictions,n)